In [26]:
from transformers import VitsModel, AutoTokenizer, AutoModelForTextToWaveform
import torch
import noisereduce as nr
import numpy as np

# model = VitsModel.from_pretrained("facebook/mms-tts-eng")
# tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-eng")

# model = VitsModel.from_pretrained("facebook/mms-tts-swh")
# tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-swh")

model = AutoModelForTextToWaveform.from_pretrained("khof312/mms-tts-swh-female-2")
tokenizer = AutoTokenizer.from_pretrained("khof312/mms-tts-swh-female-2")


In [27]:

text = "Mambo vipi?. Hii ni sauti ya Mai Sa, EI AI kujaribishwa"
inputs = tokenizer(text, return_tensors="pt")

In [28]:
with torch.no_grad():
    output = model(**inputs).waveform

In [29]:
# Convert PyTorch tensor to NumPy array
# output_np = output.squeeze().cpu().numpy()
output_np = output.detach().cpu().numpy().squeeze()
output_np = (output_np * 32767).astype(np.int16)
output_np = output_np.astype("int16")  # Convert to int16 for audio

In [30]:
import scipy
# scipy.io.wavfile.write("output.wav", rate=model.config.sampling_rate, data=output.float().numpy())
scipy.io.wavfile.write("output.wav", rate=model.config.sampling_rate, data=output_np)

from IPython.display import Audio
# Audio(output.numpy(), rate=model.config.sampling_rate)
Audio(output_np, rate=model.config.sampling_rate)
# Audio(output_np, rate=16000)


In [33]:
from pydub import AudioSegment
import numpy as np

# Load the audio file
# audio = AudioSegment.from_file("output.wav", format="wav")
# audio = output_np.astype("int16")  # Convert to int16 for audio
# output_np = (output_np * 32767).astype(np.int16)
audio = AudioSegment(
    output_np.tobytes(),
    frame_rate=16000,  # Assuming the sample rate is 16kHz
    sample_width=2,  # 16-bit audio = 2 bytes
    channels=1  # Mono audio
)

# Change the pitch by modifying the frame rate
new_sample_rate = int(audio.frame_rate * 1.0)  # Increase pitch by 20%
pitched_audio = audio._spawn(audio.raw_data, overrides={'frame_rate': new_sample_rate})

# Set the frame rate back to the original value (for playback)
pitched_audio = pitched_audio.set_frame_rate(audio.frame_rate)

# Save the pitch-shifted audio
pitched_audio.export("output_pitch_shifted_pydub.wav", format="wav")

# Convert AudioSegment to raw data (samples)
samples = np.array(pitched_audio.get_array_of_samples())

# Normalize the data to be in the range [-1, 1] for playback
normalized_samples = samples / (2**15)

# Display and play the audio in the notebook using IPython.display.Audio
from IPython.display import Audio
Audio(data=normalized_samples, rate=pitched_audio.frame_rate)

In [32]:

from io import BytesIO
import base64

# Convert the audio to bytes
buffer = BytesIO()
pitched_audio.export(buffer, format="wav")
audio_bytes = buffer.getvalue()

# Step 4: Encode the audio as base64
audio_base64 = base64.b64encode(audio_bytes).decode('utf-8')
audio_data_uri = f"data:audio/wav;base64,{audio_base64}"

audio_data_uri

'...'

In [23]:
import librosa
import soundfile as sf

# Load the generated audio file
# y, sr = librosa.load("output.wav", sr=16000)
y, sr = librosa.load("output.wav", sr=model.config.sampling_rate)

# Apply pitch shifting: n_steps > 0 increases pitch, n_steps < 0 decreases pitch
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=-2)  # Shift up by 2 semitones

# Save the pitch-shifted audio
sf.write("output_pitch_shifted.wav", y_shifted, sr)

from IPython.display import Audio
Audio(y_shifted, rate=model.config.sampling_rate)
